# COMMERCIAL REAL ESTATE - INVESTOR RECOMMENDER

### **PROBLEM STATEMENT:**

* Aiming to bridge a technological gap between Residential and Commercial Real Estate brokerage platform offerings, PROP/SWAP serves as a tool to recommend potential buyers for individual CRE properties, referencing a comprehensive international real estate investor pool to generate customized recommendations


### **ANALYSIS SECTORS:**

* Analysis evaluates the commercial RE sectors outlined below:


    * Retail - Strip Centers, Malls, Triple-Net Retail (NNN)
    * Multifamily - Rental Apartments 
    * Office - Central Business District (CBD), Suburban (SUB)
    * Hospitality - Full-Service Hotels, Limited-Service Hotels
    * Industrial - Warehouse, Logistics

# LIBRARY IMPORTS

In [ ]:
## LIBRARY IMPORTS ##

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import time

import statistics
import datetime as dt
from math import sqrt

from scipy import stats as stats
from scipy import sparse

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor, RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import silhouette_score, mean_squared_error, f1_score

import pickle

In [ ]:
## MOUNT NOTEBOOK TO GOOGLE DRIVE
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


# DATA IMPORTS

## INVESTOR INDEX

In [ ]:
### DATA IMPORTS (PRE-PROCESSED)
all_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'INVESTORS', header = 0)
mf_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'MF', header = 0)
nnn_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'NNN', header = 0)
sc_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'SC', header = 0)
mall_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'MALL', header = 0)
ss_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'SS', header = 0)
ind_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'IND', header = 0)
fs_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'FS', header = 0)
ls_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'LS', header = 0)
cbd_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'CBD', header = 0)
sub_investor_idx = pd.read_excel('/drive/My Drive/Colab Notebooks/capstone/data/investors.xlsx', sheet_name = 'SUB', header = 0)

## HISTORICAL REAL ESTATE DATA

In [ ]:
# annual CRE acquisition volume, by sector
rca_ann = pd.read_csv('/drive/My Drive/Colab Notebooks/capstone/data/all_sectors_python_ann.csv', infer_datetime_format = True, header = 0, index_col = 'Date')

# quarterly CRE acquisition volume, by sector
rca_qtr = pd.read_csv('/drive/My Drive/Colab Notebooks/capstone/data/all_sectors_python_qtr.csv', infer_datetime_format = True, header = 0, index_col = 'Date')

## PUBLIC REIT DATA

In [ ]:
reit_comps = pd.read_csv('/drive/My Drive/Colab Notebooks/capstone/data/reit_financials.csv', infer_datetime_format = True)

# ANALYSIS PARAMETERS

## VARIABLE ASSIGNMENT

In [ ]:
## VARIABLE ASSIGNMENT - STREAMLIT

sectors = ['MULTIFAMILY', 'STRIP CENTER', 'NNN RETAIL', 'MALL', 'SELF-STORAGE', 'INDUSTRIAL', 'FULL-SERVICE HOTEL', 'LIMITED-SERVICE HOTEL', 'CBD OFFICE', 'SUBURBAN OFFICE']

streamlit_outputs = ['{sector}_UNITS_PROP',	'MF_AVG_PRICE',	'MF_AVG_PPU',	'QUALITY']


## VARIABLE ASSIGNMENT - SECTORS
mf_num_cols = mf_investor_idx[['TTL_SECT','TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE',
                               'MF_VOL', 'MF_PROPS','MF_UNITS', 'MF_UNITS_PROP', 'MF_AVG_PRICE', 'MF_AVG_PPU',
                               'QUALITY','VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

sc_num_cols = sc_investor_idx[['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE',
                               'SC_VOL', 'SC_PROPS', 'SC_SF', 'SC_SF_PROP', 'SC_AVG_PRICE', 'SC_AVG_PSF',
                               'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

nnn_num_cols = nnn_investor_idx[['TTL_SECT','TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE',
                                 'NNN_VOL', 'NNN_PROPS', 'NNN_SF', 'NNN_SF_PROP', 'NNN_AVG_PRICE', 'NNN_AVG_PSF',
                                 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

mall_num_cols = mall_investor_idx[['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE',
                                   'MALL_VOL', 'MALL_PROPS', 'MALL_SF', 'MALL_SF_PROP', 'MALL_AVG_PRICE', 'MALL_AVG_PSF',
                                   'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

ss_num_cols = ss_investor_idx[['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE',
                               'SS_VOL', 'SS_PROPS', 'SS_SF', 'SS_SF_PROP', 'SS_AVG_PRICE', 'SS_AVG_PSF',
                               'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

ind_num_cols = ind_investor_idx[['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE',
                                 'IND_VOL', 'IND_PROPS', 'IND_SF', 'IND_SF_PROP', 'IND_AVG_PRICE', 'IND_AVG_PSF',
                                 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

fs_num_cols = fs_investor_idx[['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE',
                               'FS_VOL', 'FS_PROPS', 'FS_KEYS', 'FS_KEYS_PROP', 'FS_AVG_PRICE', 'FS_AVG_PPK',
                               'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

ls_num_cols = ls_investor_idx[['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE',
                               'LS_VOL', 'LS_PROPS', 'LS_KEYS', 'LS_KEYS_PROP', 'LS_AVG_PRICE', 'LS_AVG_PPK',
                               'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

cbd_num_cols = cbd_investor_idx[['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE',
                                 'CBD_VOL', 'CBD_PROPS', 'CBD_SF', 'CBD_SF_PROP', 'CBD_AVG_PRICE', 'CBD_AVG_PSF',
                                 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

sub_num_cols = sub_investor_idx[['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE',
                                 'SUB_VOL', 'SUB_PROPS', 'SUB_SF', 'SUB_SF_PROP', 'SUB_AVG_PRICE', 'SUB_AVG_PSF',
                                 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

master_num_cols = all_investor_idx[['TTL_SECT', 'MF_SECT', 'SC_SECT', 'NNN_SECT', 'MALL_SECT',
       'SS_SECT', 'IND_SECT', 'FS_SECT', 'LS_SECT', 'CBD_SECT', 'SUB_SECT',
       'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'RET_VOL', 'RET_PROPS',
       'RET_AVG_PRICE', 'RET_AVG_PSF', 'OFF_VOL', 'OFF_PROPS', 'OFF_AVG_PRICE',
       'OFF_AVG_PSF', 'TTL_SF', 'RET_SF', 'OFF_SF', 'SS_IND_SF', 'HOT_VOL',
       'HOT_KEYS', 'MF_VOL', 'SC_VOL', 'NNN_VOL', 'MALL_VOL', 'SS_VOL',
       'IND_VOL', 'FS_VOL', 'LS_VOL', 'CBD_VOL', 'SUB_VOL', 'MF_PROPS',
       'SC_PROPS', 'NNN_PROPS', 'MALL_PROPS', 'SS_PROPS', 'IND_PROPS',
       'FS_PROPS', 'LS_PROPS', 'CBD_PROPS', 'SUB_PROPS', 'MF_UNITS', 'SC_SF',
       'NNN_SF', 'MALL_SF', 'SS_SF', 'IND_SF', 'FS_KEYS', 'LS_KEYS', 'CBD_SF',
       'SUB_SF', 'MF_AVG_PRICE', 'SC_AVG_PRICE', 'NNN_AVG_PRICE',
       'MALL_AVG_PRICE', 'SS_AVG_PRICE', 'IND_AVG_PRICE', 'FS_AVG_PRICE',
       'LS_AVG_PRICE', 'CBD_AVG_PRICE', 'SUB_AVG_PRICE', 'MF_AVG_PPU',
       'SC_AVG_PSF', 'NNN_AVG_PSF', 'MALL_AVG_PSF', 'SS_AVG_PSF',
       'IND_AVG_PSF', 'FS_AVG_PPK', 'LS_AVG_PPK', 'CBD_AVG_PSF', 'SUB_AVG_PSF',
       'MF_QUALITY', 'SC_QUALITY', 'NNN_QUALITY', 'MALL_QUALITY', 'SS_QUALITY',
       'IND_QUALITY', 'FS_QUALITY', 'LS_QUALITY', 'CBD_QUALITY', 'SUB_QUALITY',
       ]]
#       'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK']]

sect_qual_scores = all_investor_idx[['MF_QUALITY', 'SC_QUALITY', 'NNN_QUALITY', 'MALL_QUALITY', 'SS_QUALITY',
                                     'IND_QUALITY', 'FS_QUALITY', 'LS_QUALITY', 'CBD_QUALITY', 'SUB_QUALITY']]

sectors = [mf_num_cols, sc_num_cols, nnn_num_cols, mall_num_cols, 
           ss_num_cols, ind_num_cols, fs_num_cols, ls_num_cols,
           cbd_num_cols, sub_num_cols]


## VARIABLE ASSIGNMENT - UTILITIES
newline = '\n'

# MODELS

### PRINCIPAL COMPONENT ANALYSIS / LINEAR REGRESSION

In [ ]:
def evaluate_sector_lr(sector):
  #sector_results = pd.DataFrame(columns = sector.columns)
  X = sector.drop(columns = 'QUALITY')
  y = sector['QUALITY']
  pf = PolynomialFeatures(degree=3)
  X_poly = pf.fit_transform(sector)
  X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)
  sc = StandardScaler()
  X_train_sc = sc.fit_transform(X_train)
  X_test_sc = sc.transform(X_test)
  pca = PCA(random_state=42)
  pca.fit(X_train_sc)
  Z_train = pca.transform(X_train_sc)
  Z_test = pca.transform(X_test_sc)
  pd.DataFrame(Z_train).describe().T
  ### REFER TO GRIDSEARCH CONDUCTED IN APPENDED SECTIONS
  pca = PCA(n_components=45, random_state=42)
  pca.fit(X_train_sc)
  lr = LinearRegression()
  Z_train = pca.transform(X_train_sc)
  Z_test = pca.transform(X_test_sc)
  lr.fit(Z_train, y_train)
  #var_exp = pca.explained_variance_ratio_
  return f"LR TRAIN: {round(lr.score(Z_train, y_train), 4)}", f"LR TEST: {round(lr.score(Z_test, y_test), 4)}", f"PCA EXPLAINED VARIANCE: {pca.explained_variance_ratio_[:5]}"

In [ ]:
print(f"MULTIFAMILY: ", evaluate_sector_lr(mf_num_cols))
print(f"STRIP CENTER: ", evaluate_sector_lr(sc_num_cols))
print(f"NNN RETAIL: ", evaluate_sector_lr(nnn_num_cols))
print(f"MALLS: ", evaluate_sector_lr(mall_num_cols))
print(f"SELF-STORAGE: ", evaluate_sector_lr(ss_num_cols))
print(f"INDUSTRIAL: ", evaluate_sector_lr(ind_num_cols))
print(f"FULL-SERVICE HOTEL: ", evaluate_sector_lr(fs_num_cols))
print(f"LIMITED-SERVICE HOTEL: ", evaluate_sector_lr(ls_num_cols))
print(f"CBD OFFICE: ", evaluate_sector_lr(cbd_num_cols))
print(f"SUBURBAN OFFICE: ", evaluate_sector_lr(sub_num_cols))

MULTIFAMILY:  ('LR TRAIN: 0.9987', 'LR TEST: 0.998', 'PCA EXPLAINED VARIANCE: [0.36937747 0.1931883  0.08670349 0.06525984 0.06367048]')
STRIP CENTER:  ('LR TRAIN: 0.9989', 'LR TEST: 0.977', 'PCA EXPLAINED VARIANCE: [0.35701098 0.19834335 0.09767679 0.07702391 0.06242996]')
NNN RETAIL:  ('LR TRAIN: 0.9992', 'LR TEST: 0.4032', 'PCA EXPLAINED VARIANCE: [0.28595139 0.19692838 0.15101111 0.10115104 0.05445393]')
MALLS:  ('LR TRAIN: 1.0', 'LR TEST: 0.9991', 'PCA EXPLAINED VARIANCE: [0.4749243  0.15203977 0.12102144 0.09766003 0.05388178]')
SELF-STORAGE:  ('LR TRAIN: 1.0', 'LR TEST: 0.9978', 'PCA EXPLAINED VARIANCE: [0.30467877 0.24183499 0.11565321 0.0785521  0.06896769]')
INDUSTRIAL:  ('LR TRAIN: 0.9989', 'LR TEST: 0.9746', 'PCA EXPLAINED VARIANCE: [0.36724661 0.15838313 0.12346691 0.09015077 0.05437059]')
FULL-SERVICE HOTEL:  ('LR TRAIN: 0.9992', 'LR TEST: 0.0807', 'PCA EXPLAINED VARIANCE: [0.28974564 0.2060365  0.14604537 0.07604248 0.05924703]')
LIMITED-SERVICE HOTEL:  ('LR TRAIN: 0.999

### PRINCIPAL COMPONENT ANALYSIS / LOGISTIC REGRESSION

In [ ]:
def evaluate_sector_log(sector):
  #sector_results = pd.DataFrame(columns = sector.columns)
  X = sector.drop(columns = 'QUALITY')
  y = sector['QUALITY']
  pf = PolynomialFeatures(degree=3)
  X_poly = pf.fit_transform(sector)
  X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)
  sc = StandardScaler()
  X_train_sc = sc.fit_transform(X_train)
  X_test_sc = sc.transform(X_test)
  pca = PCA(random_state=42)
  pca.fit(X_train_sc)
  Z_train = pca.transform(X_train_sc)
  Z_test = pca.transform(X_test_sc)
  pd.DataFrame(Z_train).describe().T
  ### REFER TO GRIDSEARCH CONDUCTED IN APPENDED SECTIONS
  pca = PCA(n_components=45, random_state=42)
  pca.fit(X_train_sc)
  log = LogisticRegression(solver='liblinear')
  Z_train = pca.transform(X_train_sc)
  Z_test = pca.transform(X_test_sc)
  log.fit(Z_train, y_train)
  #var_exp = pca.explained_variance_ratio_
  return f"LOG TRAIN: {round(log.score(Z_train, y_train), 4)}", f"LOG TEST: {round(log.score(Z_test, y_test), 4)}", f"PCA EXPLAINED VARIANCE: {pca.explained_variance_ratio_[:5]}"

In [ ]:
print(f"MULTIFAMILY: ", evaluate_sector_log(mf_num_cols))
print(f"STRIP CENTER: ", evaluate_sector_log(sc_num_cols))
print(f"NNN RETAIL: ", evaluate_sector_log(nnn_num_cols))
print(f"MALLS: ", evaluate_sector_log(mall_num_cols))
print(f"SELF-STORAGE: ", evaluate_sector_log(ss_num_cols))
print(f"INDUSTRIAL: ", evaluate_sector_log(ind_num_cols))
print(f"FULL-SERVICE HOTEL: ", evaluate_sector_log(fs_num_cols))
print(f"LIMITED-SERVICE HOTEL: ", evaluate_sector_log(ls_num_cols))
print(f"CBD OFFICE: ", evaluate_sector_log(cbd_num_cols))
print(f"SUBURBAN OFFICE: ", evaluate_sector_log(sub_num_cols))

MULTIFAMILY:  ('LOG TRAIN: 0.9457', 'LOG TEST: 0.7629', 'PCA EXPLAINED VARIANCE: [0.36937747 0.1931883  0.08670349 0.06525984 0.06367048]')
STRIP CENTER:  ('LOG TRAIN: 0.9283', 'LOG TEST: 0.6508', 'PCA EXPLAINED VARIANCE: [0.35701098 0.19834335 0.09767679 0.07702391 0.06242996]')
NNN RETAIL:  ('LOG TRAIN: 0.9295', 'LOG TEST: 0.5897', 'PCA EXPLAINED VARIANCE: [0.28595139 0.19692838 0.15101111 0.10115104 0.05445393]')
MALLS:  ('LOG TRAIN: 0.9778', 'LOG TEST: 0.5833', 'PCA EXPLAINED VARIANCE: [0.4749243  0.15203977 0.12102144 0.09766003 0.05388178]')
SELF-STORAGE:  ('LOG TRAIN: 0.9821', 'LOG TEST: 0.2857', 'PCA EXPLAINED VARIANCE: [0.30467877 0.24183499 0.11565321 0.0785521  0.06896769]')
INDUSTRIAL:  ('LOG TRAIN: 0.9333', 'LOG TEST: 0.7049', 'PCA EXPLAINED VARIANCE: [0.36724661 0.15838313 0.12346691 0.09015077 0.05437059]')
FULL-SERVICE HOTEL:  ('LOG TRAIN: 0.952', 'LOG TEST: 0.5625', 'PCA EXPLAINED VARIANCE: [0.28974564 0.2060365  0.14604537 0.07604248 0.05924703]')
LIMITED-SERVICE HOTE

### DBSCAN

In [ ]:
## DBSCAN FUNCTION
def dbscan_sector(sector):
  X = sector.drop(columns = 'QUALITY')
  y = sector['QUALITY']
  sc = StandardScaler()
  X_scaled = sc.fit_transform(sector)
  dbscan = DBSCAN(eps=2.3, min_samples=4)
  dbscan.fit(X_scaled)
  set(dbscan.labels_)
  sector['CLUSTER'] = dbscan.labels_
  return (f"SILHOUETTE SCORE: {round(silhouette_score(X_scaled, dbscan.labels_), 4)}")

In [ ]:
## WARNINGS
import warnings
warnings.simplefilter("ignore")

## DBSCAN SILHOUETTE SCORES
print(f"MULTIFAMILY: ", dbscan_sector(mf_num_cols))
print(f"STRIP CENTER: ", dbscan_sector(sc_num_cols))
print(f"NNN RETAIL: ", dbscan_sector(nnn_num_cols))
print(f"MALLS: ", dbscan_sector(mall_num_cols))
print(f"SELF-STORAGE: ", dbscan_sector(ss_num_cols))
print(f"INDUSTRIAL: ", dbscan_sector(ind_num_cols))
print(f"FULL-SERVICE HOTEL: ", dbscan_sector(fs_num_cols))
print(f"LIMITED-SERVICE HOTEL: ", dbscan_sector(ls_num_cols))
print(f"CBD OFFICE: ", dbscan_sector(cbd_num_cols))
print(f"SUBURBAN OFFICE: ", dbscan_sector(sub_num_cols))

MULTIFAMILY:  SILHOUETTE SCORE: 0.5993
STRIP CENTER:  SILHOUETTE SCORE: 0.5062
NNN RETAIL:  SILHOUETTE SCORE: 0.6437
MALLS:  SILHOUETTE SCORE: 0.5583
SELF-STORAGE:  SILHOUETTE SCORE: 0.3825
INDUSTRIAL:  SILHOUETTE SCORE: 0.6451
FULL-SERVICE HOTEL:  SILHOUETTE SCORE: 0.5331
LIMITED-SERVICE HOTEL:  SILHOUETTE SCORE: 0.5147
CBD OFFICE:  SILHOUETTE SCORE: 0.5668
SUBURBAN OFFICE:  SILHOUETTE SCORE: 0.6161


## ENSEMBLE MODELS

In [ ]:
def ensemble_sectors(sector):
  X = sector.drop(columns = 'QUALITY')
  y = sector['QUALITY']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
  # Decision Tree Regression
  dtr = DecisionTreeRegressor()
  dtr.fit(X_train, y_train)
  dtr_preds = dtr.predict(X_test)
  dtr_rmse = sqrt(mean_squared_error(y_test, dtr_preds))
  # Bagging Regression
  bag = BaggingRegressor()
  bag.fit(X_train, y_train)
  bag_preds = bag.predict(X_test)
  bag_rmse = sqrt(mean_squared_error(y_test, bag_preds))
  # Random Forest Regression
  rfr = RandomForestRegressor()
  rfr.fit(X_train, y_train)
  rfr_preds = rfr.predict(X_test)
  rfr_rmse = sqrt(mean_squared_error(y_test, rfr_preds))
  # AdaBoost Regression
  ada = AdaBoostRegressor()
  ada.fit(X_train, y_train)
  ada_preds = ada.predict(X_test)
  ada_rmse = sqrt(mean_squared_error(y_test, ada_preds))
  return print(f"DTR TRAIN: {round(dtr.score(X_train, y_train), 4)}{newline}DTR TEST: {round(dtr.score(X_test, y_test), 4)}{newline}DTR RMSE: {round((dtr_rmse), 4)}{newline}BAG TRAIN: {round(bag.score(X_train, y_train), 4)}{newline}BAG TEST: {round(bag.score(X_test, y_test), 4)}{newline}BAG RMSE: {round((bag_rmse), 4)}{newline}RFR TRAIN: {round(rfr.score(X_train, y_train), 4)}{newline}RFR TEST: {round(rfr.score(X_test, y_test), 4)}{newline}RFR RMSE: {round((rfr_rmse), 4)}{newline}ADA TRAIN: {round(ada.score(X_train, y_train), 4)}{newline}ADA TEST: {round(ada.score(X_test, y_test), 4)}{newline}ADA RMSE: {round((ada_rmse), 4)}")
  #return print(f"LR TRAIN: {round(lr.score(X_train, y_train), 4)}{newline}LR TEST: {round(lr.score(X_test, y_test), 4)}{newline}LR RMSE: {round((lr_rmse), 4)}{newline}KNN TRAIN: {round(knn.score(X_train, y_train), 4)}{newline}KNN TEST: {round(knn.score(X_test, y_test), 4)}{newline}KNN RMSE: {round((knn_rmse), 4)}{newline}DTR TRAIN: {round(dtr.score(X_train, y_train), 4)}{newline}DTR TEST: {round(dtr.score(X_test, y_test), 4)}{newline}DTR RMSE: {round((dtr_rmse), 4)}{newline}BAG TRAIN: {round(bag.score(X_train, y_train), 4)}{newline}BAG TEST: {round(bag.score(X_test, y_test), 4)}{newline}BAG RMSE: {round((bag_rmse), 4)}{newline}RFR TRAIN: {round(rfr.score(X_train, y_train), 4)}{newline}RFR TEST: {round(rfr.score(X_test, y_test), 4)}{newline}RFR RMSE: {round((rfr_rmse), 4)}{newline}ADA TRAIN: {round(ada.score(X_train, y_train), 4)}{newline}ADA TEST: {round(ada.score(X_test, y_test), 4)}{newline}ADA RMSE: {round((ada_rmse), 4)}{newline}SVR TRAIN: {round(svr.score(X_train, y_train), 4)}{newline}SVR TEST: {round(svr.score(X_test, y_test), 4)}{newline}SVR RMSE: {round((svr_rmse), 4)}")

In [ ]:
## GENERATE SCORES / RESULTS

print(f"MULTIFAMILY: ")
ensemble_sectors(mf_num_cols)
print()
print(f"STRIP CENTER: ")
ensemble_sectors(sc_num_cols)
print()
print(f"NNN RETAIL: ")
ensemble_sectors(nnn_num_cols)
print()
print(f"MALLS: ")
ensemble_sectors(mall_num_cols)
print()
print(f"SELF-STORAGE: ")
ensemble_sectors(ss_num_cols)
print()
print(f"INDUSTRIAL: ")
ensemble_sectors(ind_num_cols)
print()
print(f"FULL-SERVICE HOTEL: ")
ensemble_sectors(fs_num_cols)
print()
print(f"LIMITED-SERVICE HOTEL: ")
ensemble_sectors(ls_num_cols)
print()
print(f"CBD OFFICE: ")
ensemble_sectors(cbd_num_cols)
print()
print(f"SUBURBAN OFFICE: ")
ensemble_sectors(sub_num_cols)

## ENSEMBLE MODELS

In [ ]:
## RUN SECTORS THROUGH MODELS
def ensemble_sectors(sector):
  X = sector.drop(columns = 'QUALITY')
  y = sector['QUALITY']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
  # Decision Tree Regression
  dtr = DecisionTreeRegressor()
  dtr.fit(X_train, y_train)
  dtr_preds = dtr.predict(X_test)
  dtr_rmse = sqrt(mean_squared_error(y_test, dtr_preds))
  # Bagging Regression
  bag = BaggingRegressor()
  bag.fit(X_train, y_train)
  bag_preds = bag.predict(X_test)
  bag_rmse = sqrt(mean_squared_error(y_test, bag_preds))
  # Random Forest Regression
  rfr = RandomForestRegressor()
  rfr.fit(X_train, y_train)
  rfr_preds = rfr.predict(X_test)
  rfr_rmse = sqrt(mean_squared_error(y_test, rfr_preds))
  # AdaBoost Regression
  ada = AdaBoostRegressor()
  ada.fit(X_train, y_train)
  ada_preds = ada.predict(X_test)
  ada_rmse = sqrt(mean_squared_error(y_test, ada_preds))
  
  return print(f"DTR TRAIN: {round(dtr.score(X_train, y_train), 4)}{newline}DTR TEST: {round(dtr.score(X_test, y_test), 4)}{newline}DTR RMSE: {round((dtr_rmse), 4)}{newline}BAG TRAIN: {round(bag.score(X_train, y_train), 4)}{newline}BAG TEST: {round(bag.score(X_test, y_test), 4)}{newline}BAG RMSE: {round((bag_rmse), 4)}{newline}RFR TRAIN: {round(rfr.score(X_train, y_train), 4)}{newline}RFR TEST: {round(rfr.score(X_test, y_test), 4)}{newline}RFR RMSE: {round((rfr_rmse), 4)}{newline}ADA TRAIN: {round(ada.score(X_train, y_train), 4)}{newline}ADA TEST: {round(ada.score(X_test, y_test), 4)}{newline}ADA RMSE: {round((ada_rmse), 4)}")

In [ ]:
## GENERATE SCORES / RESULTS
print(f"MULTIFAMILY: ")
ensemble_sectors(mf_num_cols)
print()
print(f"STRIP CENTER: ")
ensemble_sectors(sc_num_cols)
print()
print(f"NNN RETAIL: ")
ensemble_sectors(nnn_num_cols)
print()
print(f"MALLS: ")
ensemble_sectors(mall_num_cols)
print()
print(f"SELF-STORAGE: ")
ensemble_sectors(ss_num_cols)
print()
print(f"INDUSTRIAL: ")
ensemble_sectors(ind_num_cols)
print()
print(f"FULL-SERVICE HOTEL: ")
ensemble_sectors(fs_num_cols)
print()
print(f"LIMITED-SERVICE HOTEL: ")
ensemble_sectors(ls_num_cols)
print()
print(f"CBD OFFICE: ")
ensemble_sectors(cbd_num_cols)
print()
print(f"SUBURBAN OFFICE: ")
ensemble_sectors(sub_num_cols)

## SCALED MODELS

In [ ]:
## RUN SECTORS THROUGH MODELS
def not_ensemble_sectors(sector):
  X = sector.drop(columns = 'QUALITY')
  y = sector['QUALITY']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
  # Standard Scaler
  sc = StandardScaler()
  X_train = sc.fit_transform(X_train)
  X_test = sc.transform(X_test)
  # Linear Regression
  lr = LinearRegression()
  lr.fit(X_train, y_train)
  lr_preds = lr.predict(X_test)
  lr_rmse = sqrt(mean_squared_error(y_test, lr_preds))
  # KNN Regression
  knn = KNeighborsRegressor()
  knn.fit(X_train, y_train)
  knn_preds = knn.predict(X_test)
  knn_rmse = sqrt(mean_squared_error(y_test, knn_preds))
  # Support Vector Regression
  svr = SVR()
  svr.fit(X_train, y_train)
  svr_preds = svr.predict(X_test)
  svr_rmse = sqrt(mean_squared_error(y_test, svr_preds))
  return print(f"LR TRAIN: {round(lr.score(X_train, y_train), 4)}{newline}LR TEST: {round(lr.score(X_test, y_test), 4)}{newline}LR RMSE: {round((lr_rmse), 4)}{newline}KNN TRAIN: {round(knn.score(X_train, y_train), 4)}{newline}KNN TEST: {round(knn.score(X_test, y_test), 4)}{newline}KNN RMSE: {round((knn_rmse), 4)}{newline}SVR TRAIN: {round(svr.score(X_train, y_train), 4)}{newline}SVR TEST: {round(svr.score(X_test, y_test), 4)}{newline}SVR RMSE: {round((svr_rmse), 4)}")

In [ ]:
## GENERATE SCORES / RESULTS
print(f"MULTIFAMILY: ")
not_ensemble_sectors(mf_num_cols)
print()
print(f"STRIP CENTER: ")
not_ensemble_sectors(sc_num_cols)
print()
print(f"NNN RETAIL: ")
not_ensemble_sectors(nnn_num_cols)
print()
print(f"MALLS: ")
not_ensemble_sectors(mall_num_cols)
print()
print(f"SELF-STORAGE: ")
not_ensemble_sectors(ss_num_cols)
print()
print(f"INDUSTRIAL: ")
not_ensemble_sectors(ind_num_cols)
print()
print(f"FULL-SERVICE HOTEL: ")
not_ensemble_sectors(fs_num_cols)
print()
print(f"LIMITED-SERVICE HOTEL: ")
not_ensemble_sectors(ls_num_cols)
print()
print(f"CBD OFFICE: ")
not_ensemble_sectors(cbd_num_cols)
print()
print(f"SUBURBAN OFFICE: ")
not_ensemble_sectors(sub_num_cols)

## VOTING CLASSIFIER

In [ ]:
## SECTOR FUNCTION
# def template_func(sector):

X = mf_num_cols.drop(columns = 'QUALITY')
y = mf_num_cols['QUALITY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
## VOTING CLASSIFIER
vc = VotingClassifier([
  ('ada', AdaBoostClassifier()),
  ('gbc', GradientBoostingClassifier()),
  ('rfr', RandomForestClassifier())
  ])

## VOTING CLASSIFIER hyperparameter grid
vc_params = {
  'ada__base_estimator': [None, DecisionTreeClassifier(max_depth=2, random_state=42)],
  'ada__n_estimators': [50],
  'gbc__n_estimators': [50],
  'rfr__max_depth': [10],
  'rfr__min_samples_split': [2]
  }

In [ ]:
vc_grid = GridSearchCV(vc, param_grid=vc_params, cv=5, n_jobs=8)

In [ ]:
#%%time
grid.fit(X_train, y_train)
print(grid.best_score_)
grid.best_params_

0.9876923076923078


{'ada__base_estimator': None,
 'ada__n_estimators': 50,
 'dtc__max_depth': 5,
 'gbc__n_estimators': 50,
 'rfr__max_depth': 9,
 'rfr__min_samples_split': 2}

* Iterating through voting classifier hyperparameters to determine optimal weighting for each ensemble model
  * Computationally expensive; highly tactical gridsearch to achieve improved performance

# BUYER RECOMMENDER

In [ ]:
mf_num_cols.describe()

,TTL_SECT,TTL_VOL,TTL_PROPS,TTL_AVG_PRICE,MF_VOL,MF_PROPS,MF_UNITS,MF_UNITS_PROP,MF_AVG_PRICE,MF_AVG_PPU,QUALITY,VOL_RANK,PRICE_UNIT_RANK,UNIT_RANK
count,484.000000,4.840000e+02,484.000000,4.840000e+02,4.840000e+02,484.000000,484.000000,484.000000,4.840000e+02,484.000000,484.000000,484.000000,484.000000,484.000000
mean,1.646694,8.139529e+08,16.851240,5.703495e+07,4.872665e+08,10.849174,2590.876033,270.631181,5.848182e+07,219739.904988,5.530992,242.508264,246.489669,242.508264
std,1.429202,2.153957e+09,41.364543,4.495334e+07,6.745966e+08,13.015359,3352.809176,148.376485,4.525344e+07,125511.784363,2.136412,139.923678,139.869334,139.923678
min,1.000000,1.351500e+08,1.000000,3.249336e+06,1.351500e+08,1.000000,232.000000,18.688889,3.249336e+06,53985.476012,1.000000,1.000000,4.000000,1.000000
25%,1.000000,1.931458e+08,5.000000,2.944787e+07,1.822489e+08,4.000000,998.750000,204.150000,3.016250e+07,133498.395348,4.000000,121.750000,125.750000,121.750000
50%,1.000000,3.239977e+08,9.000000,4.548018e+07,2.812000e+08,7.000000,1642.500000,260.684615,4.611970e+07,184380.040826,5.000000,242.500000,246.500000,242.500000
75%,1.000000,7.516109e+08,15.000000,6.639135e+07,5.259863e+08,13.000000,2824.750000,316.934783,6.934188e+07,273486.841434,7.000000,363.250000,367.250000,363.250000
max,9.000000,3.181488e+10,716.000000,4.149636e+08,7.510135e+09,114.000000,37502.000000,1675.000000,3.131955e+08,764421.394889,10.000000,487.000000,488.000000,487.000000


In [ ]:
## STREAMLIT BUYER SCREEN

def filter_buyers(sector, prop_size, min_prop_price, prop_qual):
  if sector == 'MULTIFAMILY':
    for investors in mf_investor_idx:
      mf_size_filter = mf_investor_idx[mf_investor_idx['MF_UNITS_PROP'] >= prop_size]
      mf_min_price_filter = mf_size_filter[mf_size_filter['MF_AVG_PRICE'] >= min_prop_price]
      mf_qual_filter = mf_min_price_filter[(mf_min_price_filter['QUALITY'] >= (prop_qual-1)) & (mf_min_price_filter['QUALITY'] <= (prop_qual+1))]
      mf_buyer_recs = mf_qual_filter.sort_values(by = 'UNIT_RANK', ascending = True)[:10]
      mf_buyer_recs.drop(axis = 1, columns = ['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK', 'MF_VOL', 'MF_PROPS', 'MF_UNITS'], inplace = True)
    return mf_buyer_recs
  elif sector == 'STRIP CENTER':
    for investors in sc_investor_idx:
      sc_size_filter = sc_investor_idx[sc_investor_idx['SC_SF_PROP'] >= prop_size]
      sc_min_price_filter = sc_size_filter[sc_size_filter['SC_AVG_PRICE'] >= min_prop_price]
      sc_qual_filter = sc_min_price_filter[(sc_min_price_filter['QUALITY'] >= (prop_qual-1)) & (sc_min_price_filter['QUALITY'] <= (prop_qual+1))]
      sc_buyer_recs = sc_qual_filter.sort_values(by = 'UNIT_RANK', ascending = True)[:10]
      sc_buyer_recs.drop(axis = 1, columns = ['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK', 'SC_VOL', 'SC_PROPS', 'SC_SF'], inplace = True)
    return sc_buyer_recs
  elif sector == 'NNN RETAIL':
    for investors in nnn_investor_idx:
      nnn_size_filter = nnn_investor_idx[nnn_investor_idx['NNN_SF_PROP'] >= prop_size]
      nnn_min_price_filter = nnn_size_filter[nnn_size_filter['NNN_AVG_PRICE'] >= min_prop_price]
      nnn_qual_filter = nnn_min_price_filter[(nnn_min_price_filter['QUALITY'] >= (prop_qual-1)) & (nnn_min_price_filter['QUALITY'] <= (prop_qual+1))]
      nnn_buyer_recs = nnn_qual_filter.sort_values(by = 'UNIT_RANK', ascending = True)[:10]
      nnn_buyer_recs.drop(axis = 1, columns = ['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK', 'NNN_VOL', 'NNN_PROPS', 'NNN_SF'], inplace = True)
    return nnn_buyer_recs
  elif sector == 'MALL':
    for investors in mall_investor_idx:
      mall_size_filter = mall_investor_idx[mall_investor_idx['MALL_SF_PROP'] >= prop_size]
      mall_min_price_filter = mall_size_filter[mall_size_filter['MALL_AVG_PRICE'] >= min_prop_price]
      mall_qual_filter = mall_min_price_filter[(mall_min_price_filter['QUALITY'] >= (prop_qual-2)) & (mall_min_price_filter['QUALITY'] <= (prop_qual+2))]
      mall_buyer_recs = mall_qual_filter.sort_values(by = 'MALL_PROPS', ascending = False)[:10]
      mall_buyer_recs.drop(axis = 1, columns = ['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK', 'MALL_VOL', 'MALL_PROPS', 'MALL_SF'], inplace = True)
    return mall_buyer_recs
  elif sector == 'SELF-STORAGE':
    for investors in ss_investor_idx:
      ss_size_filter = ss_investor_idx[ss_investor_idx['SS_SF_PROP'] >= prop_size]
      ss_min_price_filter = ss_size_filter[ss_size_filter['SS_AVG_PRICE'] >= min_prop_price]
      ss_qual_filter = ss_min_price_filter[(ss_min_price_filter['QUALITY'] >= (prop_qual-1)) & (ss_min_price_filter['QUALITY'] <= (prop_qual+1))]
      ss_buyer_recs = ss_qual_filter.sort_values(by = 'UNIT_RANK', ascending = True)[:10]
      ss_buyer_recs.drop(axis = 1, columns = ['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK', 'SS_VOL', 'SS_PROPS', 'SS_SF'], inplace = True)
    return ss_buyer_recs
  elif sector == 'INDUSTRIAL':
    for investors in ind_investor_idx:
      ind_size_filter = ind_investor_idx[ind_investor_idx['IND_SF_PROP'] >= prop_size]
      ind_min_price_filter = ind_size_filter[ind_size_filter['IND_AVG_PRICE'] >= min_prop_price]
      ind_qual_filter = ind_min_price_filter[(ind_min_price_filter['QUALITY'] >= (prop_qual-1)) & (ind_min_price_filter['QUALITY'] <= (prop_qual+1))]
      ind_buyer_recs = ind_qual_filter.sort_values(by = 'UNIT_RANK', ascending = True)[:10]
      ind_buyer_recs.drop(axis = 1, columns = ['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK', 'IND_VOL', 'IND_PROPS', 'IND_SF'], inplace = True)
    return ind_buyer_recs
  elif sector == 'FULL-SERVICE HOTEL':
    for investors in fs_investor_idx:
      fs_size_filter = fs_investor_idx[fs_investor_idx['FS_SF_PROP'] >= prop_size]
      fs_min_price_filter = fs_size_filter[fs_size_filter['FS_AVG_PRICE'] >= min_prop_price]
      fs_qual_filter = fs_min_price_filter[(fs_min_price_filter['QUALITY'] >= (prop_qual-1)) & (fs_min_price_filter['QUALITY'] <= (prop_qual+1))]
      fs_buyer_recs = fs_qual_filter.sort_values(by = 'UNIT_RANK', ascending = True)[:10]
      fs_buyer_recs.drop(axis = 1, columns = ['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK', 'FS_VOL', 'FS_PROPS', 'FS_KEYS'], inplace = True)
    return fs_buyer_recs
  elif sector == 'LIMITED-SERVICE HOTEL':
    for investors in ls_investor_idx:
      ls_size_filter = ls_investor_idx[ls_investor_idx['LS_KEYS_PROP'] >= prop_size]
      ls_min_price_filter = ls_size_filter[ls_size_filter['LS_AVG_PRICE'] >= min_prop_price]
      ls_qual_filter = ls_min_price_filter[(ls_min_price_filter['QUALITY'] >= (prop_qual-1)) & (ls_min_price_filter['QUALITY'] <= (prop_qual+1))]
      ls_buyer_recs = ls_qual_filter.sort_values(by = 'UNIT_RANK', ascending = True)[:10]
      ls_buyer_recs.drop(axis = 1, columns = ['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK', 'LS_VOL', 'LS_PROPS', 'LS_KEYS'], inplace = True)
    return ls_buyer_recs
  elif sector == 'CBD OFFICE':
    for investors in cbd_investor_idx:
      cbd_size_filter = cbd_investor_idx[cbd_investor_idx['CBD_SF_PROP'] >= prop_size]
      cbd_min_price_filter = cbd_size_filter[cbd_size_filter['CBD_AVG_PRICE'] >= min_prop_price]
      cbd_qual_filter = cbd_min_price_filter[(cbd_min_price_filter['QUALITY'] >= (prop_qual-1)) & (cbd_min_price_filter['QUALITY'] <= (prop_qual+1))]
      cbd_buyer_recs = cbd_qual_filter.sort_values(by = 'UNIT_RANK', ascending = True)[:10]
      cbd_buyer_recs.drop(axis = 1, columns = ['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK', 'CBD_VOL', 'CBD_PROPS', 'CBD_SF'], inplace = True)
    return cbd_buyer_recs
  elif sector == 'SUB OFFICE':
    for investors in sub_investor_idx:
      sub_size_filter = sub_investor_idx[sub_investor_idx['SUB_SF_PROP'] >= prop_size]
      sub_min_price_filter = sub_size_filter[sub_size_filter['SUB_AVG_PRICE'] >= min_prop_price]
      sub_qual_filter = sub_min_price_filter[(sub_min_price_filter['QUALITY'] >= (prop_qual-1)) & (sub_min_price_filter['QUALITY'] <= (prop_qual+1))]
      sub_buyer_recs = sub_qual_filter.sort_values(by = 'UNIT_RANK', ascending = True)[:10]
      sub_buyer_recs.drop(axis = 1, columns = ['TTL_SECT', 'TTL_VOL', 'TTL_PROPS', 'TTL_AVG_PRICE', 'QUALITY', 'VOL_RANK', 'PRICE_UNIT_RANK', 'UNIT_RANK', 'SUB_VOL', 'SUB_PROPS', 'SUB_SF'], inplace = True)
    return sub_buyer_recs

In [ ]:
## USER INPUTS ##
prop_units = ['UNITS', 'SF', 'KEYS']

sector = 'MALL'
prop_size = 5000
min_prop_price = 50000000
prop_qual = 2

buyer_recs = filter_buyers(sector, prop_size, min_prop_price, prop_qual)
buyer_recs.head(10)

,INVESTOR,INV_TYPE,CITY,STATE,COUNTRY,MSA,WEBSITE,C_SUITE,MALL_SF_PROP,MALL_AVG_PRICE,MALL_AVG_PSF
40,Taubman,PUBLIC REIT,Bloomfield Hills,MI,United States,Detroit,http://www.taubman.com,Robert S Taubman,1548053.0,106251375.0,68.635489
42,Cypress Equities,DEV. / OWN.,Dallas,TX,United States,Dallas,http://www.cypressequities.com,Chris Maguire,938000.0,60338585.3,64.326850


In [ ]:
buyer_recs.head(10)

,INVESTOR,INV_TYPE,CITY,STATE,COUNTRY,MSA,WEBSITE,C_SUITE,TTL_SECT,TTL_VOL,TTL_PROPS,TTL_AVG_PRICE,CBD_VOL,CBD_PROPS,CBD_SF,CBD_SF_PROP,CBD_AVG_PRICE,CBD_AVG_PSF,QUALITY,VOL_RANK,PRICE_UNIT_RANK,UNIT_RANK
155,Brookfield AM,INV. MGR.,Toronto,ON,Canada,Toronto,http://www.brookfield.com,Barry Blattman/Ric Clark/Bruce Flatt,9,3.181488e+10,348,9.142206e+07,5.776752e+09,29,12579938,433790.965517,1.991983e+08,459.203496,6,1,165,1
111,Beacon Capital Partners,EQUITY FUND,Boston,MA,United States,Boston Metro,http://www.beaconcapital.com,Alan M Leventhal,2,3.885604e+09,16,2.428503e+08,3.202450e+09,10,5760350,576035.000000,3.202450e+08,555.947071,6,2,121,2
121,Blackstone,EQUITY FUND,New York,NY,United States,NYC Metro,http://www.blackstone.com,Stephen Schwarzman/Jon Gray,9,2.823992e+10,716,3.944122e+07,2.430002e+09,12,4701579,391798.250000,2.025001e+08,516.847987,6,3,131,3
246,Columbia Property Trust,PUBLIC REIT,Sandy Springs,GA,United States,Atlanta,http://www.columbiapropertytrust.com,E Nelson Mills,3,1.139982e+09,31,3.677360e+07,1.033493e+09,17,4406136,259184.470588,6.079371e+07,234.557702,4,4,256,4
221,David Werner RE,DEV. / OWN.,New York,NY,United States,NYC Metro,00:00:00,David Werner,1,1.048700e+09,5,2.097400e+08,1.048700e+09,5,3561066,712213.200000,2.097400e+08,294.490423,4,6,231,6
171,Hines,INV. MGR.,Houston,TX,United States,Houston,http://www.hines.com,Jeffrey C Hines/Gerald D Hines,7,3.006387e+09,20,1.503194e+08,1.390837e+09,5,3489578,697915.600000,2.781675e+08,398.568935,5,7,181,7
109,Silverstein Properties,DEV. / OWN.,New York,NY,United States,NYC Metro,http://www.silversteinproperties.com,Larry A Silverstein/Marty S Burger,1,1.604099e+09,5,3.208198e+08,1.604099e+09,5,2839575,567915.000000,3.208198e+08,564.908050,6,9,119,9
220,Prologis,PUBLIC REIT,San Francisco,CA,United States,SF Metro,http://www.prologis.com,Hamid R. Moghadam,6,1.234925e+10,389,3.174615e+07,8.388985e+08,4,2798104,699526.000000,2.097246e+08,299.809619,4,10,230,10
181,SL Green,PUBLIC REIT,New York,NY,United States,NYC Metro,http://www.slgreen.com,Marc Holliday,2,1.129030e+09,8,1.411287e+08,9.861200e+08,5,2689460,537892.000000,1.972240e+08,366.660965,5,12,191,12
117,JP Morgan,INV. MGR.,New York,NY,United States,NYC Metro,https://www.jpmorgan.com/country/US/en/jpmorgan,Jamie Dimon,5,2.438869e+09,34,7.173144e+07,1.428900e+09,4,2609982,652495.500000,3.572250e+08,547.475040,6,15,127,15


# PICKLING

In [ ]:
## VARIABLE ASSIGNMENT - STREAMLIT

prop_sector_dropdown = ['MULTIFAMILY', 'STRIP CENTER', 'NNN RETAIL', 'MALL', 'SELF-STORAGE', 'INDUSTRIAL', 'FULL-SERVICE HOTEL', 'LIMITED-SERVICE HOTEL', 'CBD OFFICE', 'SUBURBAN OFFICE']

prop_measure_dropdown = ['UNITS', 'SF', 'KEYS']
  #prop_size_dropdown = {['UNITS']: list(range(10,1010,10)), ['SF']: list(range(1000,1001000,1000)), ['KEYS']: list(range(10,1010,10))}
prop_mf_unit_dropdown = list(range(10,1010,10))
prop_sf_dropdown = list(range(1000,1001000,1000))
prop_key_dropdown = list(range(10,1010,10))
prop_qual_dropdown = list(range(1,11,1))
min_price_dropdown = list(range(1000000,501000000,1000000))
  #prop_qual_dropdown = frozenset(range(30))

#filter_buyers(sector, prop_size, min_prop_price, prop_qual)
streamlit_outputs = ['{sector}_UNITS_PROP',	'MF_AVG_PRICE',	'MF_AVG_PPU',	'QUALITY']

In [ ]:
## PICKLE BUYER RECOMMENDER
with open('/drive/My Drive/Colab Notebooks/capstone/pickle.pkl', 'wb') as pickle_out:
    pickle_out = pickle.dump(filter_buyers(sector, prop_size, min_prop_price, prop_qual), pickle_out)

In [ ]:
with open('/drive/My Drive/Colab Notebooks/capstone/pickle.pkl', 'rb') as pickle_in:
    buyer_recs = pickle.load(pickle_in)

In [ ]:
buyer_recs

,INVESTOR,INV_TYPE,CITY,STATE,COUNTRY,MSA,WEBSITE,C_SUITE,TTL_SECT,TTL_VOL,TTL_PROPS,TTL_AVG_PRICE,SUB_VOL,SUB_PROPS,SUB_SF,SUB_SF_PROP,SUB_AVG_PRICE,SUB_AVG_PSF,QUALITY,VOL_RANK,PRICE_UNIT_RANK,UNIT_RANK
201,Office Props Income Trust,PUBLIC REIT,Newton,MA,United States,Boston Metro,https://www.opireit.com/home/default.aspx,David Blackman,3,3.326432e+09,62,5.365212e+07,2.849545e+09,50,10245680.0,204913.600000,5.699091e+07,278.121645,6,1,206,1
351,Bridge Investment Grp,EQUITY FUND,Sandy,UT,United States,Salt Lake City,http://www.bridge-igp.com/,Jonathan Slager/Christian Young,3,4.221328e+09,89,4.743065e+07,1.394803e+09,43,8084721.0,188016.767442,3.243729e+07,172.523391,4,2,356,2
347,DRA Advisors,INV. MGR.,New York,NY,United States,NYC Metro,http://www.draadvisors.com,David Luski,5,3.002243e+09,91,3.299168e+07,1.158425e+09,27,6615384.0,245014.222222,4.290461e+07,175.110709,4,4,352,4
341,TPG Real Estate,EQUITY FUND,Fort Worth,TX,United States,Dallas,http://www.tpg.com/,Jon Winkelried/Jim Coulter,3,1.544457e+09,48,3.217620e+07,1.090252e+09,40,6101572.0,152539.300000,2.725631e+07,178.683828,4,5,346,5
252,Boyd Watterson,INV. MGR.,Cleveland,OH,United States,Cleveland,http://www.boydwatterson.com/,Robert Law,2,1.942070e+09,55,3.531037e+07,1.207071e+09,46,5054958.0,109890.391304,2.624068e+07,238.789561,5,7,257,7
216,Partners Group,INV. MGR.,Baar,Zug,Switzerland,Zurich,https://www.partnersgroup.com/en/,Andre Frei/David Layton,2,1.479549e+09,13,1.138115e+08,1.312199e+09,11,4926296.0,447845.090909,1.192908e+08,266.366254,6,8,221,8
280,PCCP (Pacific Coast),EQUITY FUND,Los Angeles,CA,United States,LA Metro,http://www.pccpllc.com,Donald H Kuemmeler/William R Lindsay/Aaron Gio...,5,2.030852e+09,34,5.973095e+07,1.021999e+09,22,4669686.0,212258.454545,4.645450e+07,218.858197,5,10,285,10
145,MB Real Estate,DEV. / OWN.,Chicago,IL,United States,Chicago,http://www.mbres.com,Peter E Ricker,3,1.414248e+09,97,1.457987e+07,1.313915e+09,93,4116384.0,44262.193548,1.412812e+07,319.191643,6,12,150,12
185,Lincoln Property Co,DEV. / OWN.,Dallas,TX,United States,Dallas,http://www.lpc.com,Mack Pogue,4,2.074775e+09,47,4.414414e+07,1.173948e+09,25,4037107.0,161484.280000,4.695792e+07,290.789439,6,13,190,13
274,Blackstone,EQUITY FUND,New York,NY,United States,NYC Metro,http://www.blackstone.com,Stephen Schwarzman/Jon Gray,9,2.823992e+10,716,3.944122e+07,8.432940e+08,32,3748321.0,117135.031250,2.635294e+07,224.979121,5,15,279,15
